# CONFIGURATION

In [ ]:
import os

MODEL_NAMES = [
    "joeddav/xlm-roberta-large-xnli",  # (xong) lr: 6e-06
    "microsoft/infoxlm-large",  # (xong) lr: 6e-6
    "uitnlp/CafeBERT",  # (xong) lr: 2e-5
    "FacebookAI/xlm-roberta-large",  # (xong) lr: 6e-6
    "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli",  # (xong) lr: 6e-6
    "MoritzLaurer/ernie-m-large-mnli-xnli",
    "microsoft/deberta-xlarge-mnli",
]


class Config:
    ROOT_DIR = os.getcwd()
    # --- Đường dẫn và Tên file ---
    DATA_DIR = os.path.join(ROOT_DIR, "data")
    TRAIN_FILE = os.path.join(DATA_DIR, "vihallu-train.csv")
    TEST_FILE = os.path.join(DATA_DIR, "vihallu-public-test.csv")

    SUBMISSION_DIR = os.path.join(ROOT_DIR, "submission")
    SUBMISSION_CSV = "submit.csv"
    SUBMISSION_ZIP = "submit.zip"

    MODEL_NAME = MODEL_NAMES[5]  # Doi index de chon model
    MODEL_OUTPUT_DIR = os.path.join(
        ROOT_DIR, "models", f"{MODEL_NAME.split('/')[-1]}-tuned"
    )

    # --- Cấu hình Tokenizer ---
    MAX_LENGTH = 512

    # --- Cấu hình Huấn luyện ---
    RANDOM_STATE = 42
    EPOCHS = 10
    BATCH_SIZE = 2
    GRADIENT_ACCUMULATION_STEPS = 8  # Tăng dần để phù hợp với GPU VRAM

    SCHEDULER_TYPE = "cosine"  # "linear"  # Chọn scheduler phù hợp
    LEARNING_RATE = 1e-5
    WEIGHT_DECAY = 0.03
    CLASSIFIER_DROPOUT = 0.02
    EPSILON = 1e-8
    PATIENCE_LIMIT = 2
    TOTAL_STEP_SCALE = 0.1  # Sử dụng số bước để warm-up
    LABEL_SMOOTHING = 0.05  # Thêm để regularize và tránh overfitting
    VALIDATION_SPLIT_SIZE = 0.2

    # --- Ánh xạ Nhãn ---
    LABEL_MAP = {
        "intrinsic": 0,
        "extrinsic": 1,
        "no": 2,
    }  # contradiction/neutral/entailment
    ID2LABEL = {v: k for k, v in LABEL_MAP.items()}

    # << THÊM DÒNG NÀY (sử dụng con số bạn tính được từ EDA)
    CLASS_WEIGHTS = [1.0393466963622866, 1.0114145354717525, 0.9531590413943355]
    # CLASS_WEIGHTS = None


cfg = Config()


# LOGGER

In [ ]:
import logging
import os
from datetime import datetime

# Thư mục gốc để lưu tất cả các file log
LOG_BASE_DIR = "logs"

# Dùng một dictionary để lưu các logger đã tạo, tránh việc tạo lại và gây ra log trùng lặp
_loggers = {}


def setup_logger(model_name: str, log_level=logging.INFO):
    """
    Thiết lập và trả về một logger để ghi log vào cả console và file.

    - Mỗi model sẽ có một thư mục log riêng dựa trên `model_name`.
    - Mỗi lần chạy sẽ tạo một file log mới có tên là timestamp (ví dụ: 2023-10-27_15-30-00.log).
    - Đảm bảo không có log nào bị ghi đè.

    Args:
        model_name (str): Tên của model, dùng để tạo thư mục con. Ví dụ: 'xnli-large-tuned'.
        log_level (int): Cấp độ log, mặc định là logging.INFO.

    Returns:
        logging.Logger: Instance của logger đã được cấu hình.
    """
    # Nếu logger cho model này đã tồn tại, trả về nó ngay lập tức
    if model_name in _loggers:
        return _loggers[model_name]

    # Xử lý tên model để an toàn khi tạo tên thư mục (thay thế "/")
    safe_model_name = model_name.replace("/", "_").replace("\\", "_")
    model_log_dir = os.path.join(LOG_BASE_DIR, safe_model_name)
    os.makedirs(model_log_dir, exist_ok=True)

    # Tạo logger
    logger = logging.getLogger(safe_model_name)
    logger.setLevel(log_level)

    # Ngăn không cho log lan truyền đến root logger để tránh in ra console 2 lần
    logger.propagate = False

    # Định dạng cho log message
    formatter = logging.Formatter(
        "%(asctime)s - [%(levelname)s] - %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
    )

    # Tạo File Handler để ghi log ra file
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_file_path = os.path.join(model_log_dir, f"{timestamp}.log")

    file_handler = logging.FileHandler(log_file_path, encoding="utf-8")
    file_handler.setLevel(log_level)
    file_handler.setFormatter(formatter)

    # Tạo Console (Stream) Handler để in log ra màn hình
    console_handler = logging.StreamHandler()
    console_handler.setLevel(log_level)
    console_handler.setFormatter(formatter)

    # Thêm các handler vào logger
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    # Lưu logger vào cache
    _loggers[model_name] = logger

    logger.info(
        f"Logger cho '{safe_model_name}' đã được khởi tạo. File log: {log_file_path}"
    )

    return logger


## Setup logger

In [3]:
logger = setup_logger(f"{cfg.MODEL_NAME}-training")
logger.info(f"Logger initialized for {cfg.MODEL_NAME}")

logger.info("=" * 60)
logger.info("🚀 STARTING TRAINING SESSION")
logger.info("=" * 60)
for key, value in Config.__dict__.items():
    if not key.startswith("__") and not callable(value):
        logger.info(f"{key}: {value}")
logger.info("=" * 60)


2025-10-16 15:20:51 - [INFO] - Logger cho 'MoritzLaurer_ernie-m-large-mnli-xnli-training' đã được khởi tạo. File log: logs/MoritzLaurer_ernie-m-large-mnli-xnli-training/2025-10-16_15-20-51.log
2025-10-16 15:20:51 - [INFO] - Logger initialized for MoritzLaurer/ernie-m-large-mnli-xnli
2025-10-16 15:20:51 - [INFO] - ============================================================
2025-10-16 15:20:51 - [INFO] - 🚀 STARTING TRAINING SESSION
2025-10-16 15:20:51 - [INFO] - ============================================================
2025-10-16 15:20:51 - [INFO] - ROOT_DIR: /home/guest/Projects/CS221
2025-10-16 15:20:51 - [INFO] - DATA_DIR: /home/guest/Projects/CS221/data
2025-10-16 15:20:51 - [INFO] - TRAIN_FILE: /home/guest/Projects/CS221/data/vihallu-train.csv
2025-10-16 15:20:51 - [INFO] - TEST_FILE: /home/guest/Projects/CS221/data/vihallu-public-test.csv
2025-10-16 15:20:51 - [INFO] - SUBMISSION_DIR: /home/guest/Projects/CS221/submission
2025-10-16 15:20:51 - [INFO] - SUBMISSION_CSV: submit.cs

# Hallucination Dataset

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import os


# Nhiệm vụ của nó bây giờ chỉ là lấy ra các item đã được xử lý sẵn
class PreTokenizedDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Lấy tất cả các encoding cho item thứ idx
        # items() trả về một cặp (key, value)
        # ví dụ key='input_ids', value=một list chứa tất cả input_ids
        # val[idx] sẽ lấy ra input_ids cho mẫu thứ idx
        item = {key: val[idx] for key, val in self.encodings.items()}

        # Thêm nhãn vào item
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


## Prepare data

In [ ]:
def prepare_data(config, logger=None):
    """
    Đọc, tiền xử lý, chia dữ liệu thành tập train/validation và LƯU chúng ra file.
    Trả về: train_df, val_df
    """
    if os.path.isabs(getattr(config, "TRAIN_FILE", "")) or os.path.exists(
        getattr(config, "TRAIN_FILE", "")
    ):
        data_path = config.TRAIN_FILE
    else:
        data_path = os.path.join(config.DATA_DIR, config.TRAIN_FILE)

    try:
        df = pd.read_csv(data_path)
        print(f"✅ Đọc thành công {len(df)} mẫu từ {data_path}")
    except FileNotFoundError:
        print(f"❌ Lỗi: Không tìm thấy file dữ liệu tại {data_path}")
        return None, None

    # Chuyển đổi các cột sang string để tránh lỗi khi có giá trị NaN
    df["context"] = df["context"].astype(str)
    df["prompt"] = df["prompt"].astype(str)
    df["response"] = df["response"].astype(str)

    # df["input_text"] = (
    #     df["prompt"] + " </s></s> " + df["response"] + " </s></s> " + df["context"]
    # )

    # ( # <-- SỬA ĐỔI ) Áp dụng Template NLI chuẩn (Context-First)
    # -----------------------------------------------------------------
    # Gợi ý: Đây là định dạng hiệu quả cho các mô hình NLI.
    # Cấu trúc: [Premise] </s></s> [Hypothesis]

    # Ghép Prompt và Context thành nguồn chân lý (premise) -> Thêm .astype(str) để tăng độ ổn định
    premise = (
        "Câu hỏi: "
        + df["prompt"].astype(str)
        + " Ngữ cảnh: "
        + df["context"].astype(str)
    )

    # Response là giả thuyết (hypothesis) cần kiểm chứng
    hypothesis = df["response"].astype(str)

    # Tạo cột input_text cuối cùng
    df["input_text"] = premise + " </s></s> " + hypothesis
    # -----------------------------------------------------------------

    # In một vài ví dụ để kiểm tra
    print("\n=== KIỂM TRA FORMAT DỮ LIỆU MỚI ===")
    sample = df["input_text"].iloc[0]
    print(f"Mẫu input: {sample}...")

    # Ánh xạ nhãn theo logic NLI mới
    df["label_id"] = df["label"].map(config.LABEL_MAP)

    # Xử lý các dòng có thể có nhãn null sau khi map
    df.dropna(subset=["label_id"], inplace=True)
    df["label_id"] = df["label_id"].astype(int)

    # Chia train/validation
    train_df, val_df = train_test_split(
        df,
        test_size=config.VALIDATION_SPLIT_SIZE,
        random_state=config.RANDOM_STATE,
        stratify=df["label_id"],  # Đảm bảo phân bổ nhãn đều
    )

    if logger:
        logger.info(
            f"Chia dữ liệu: {len(train_df)} mẫu train, {len(val_df)} mẫu validation."
        )

    # --- PHẦN NÂNG CẤP: LƯU FILE RA THƯ MỤC DATA ---
    # Tạo thư mục 'processed' trong 'data' nếu chưa có
    processed_data_dir = os.path.join(config.DATA_DIR, "processed")
    os.makedirs(processed_data_dir, exist_ok=True)

    # Định nghĩa đường dẫn file
    train_output_path = os.path.join(processed_data_dir, "train_split.csv")
    val_output_path = os.path.join(processed_data_dir, "validation_split.csv")

    # Lưu các DataFrame
    train_df.to_csv(train_output_path, index=False, encoding="utf-8-sig")
    val_df.to_csv(val_output_path, index=False, encoding="utf-8-sig")

    print(f"✅ Đã lưu tập train vào: {train_output_path}")
    print(f"✅ Đã lưu tập validation vào: {val_output_path}")
    # --- KẾT THÚC PHẦN NÂNG CẤP ---

    return train_df, val_df


# Model

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification
import torch.nn as nn


def get_model_and_tokenizer(config):
    """Tải pre-trained model và tokenizer."""
    print(f"Đang tải model: {config.MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME)

    # Tải config/model/tokenizer với trust_remote_code=True để cho phép model custom
    cfg = AutoConfig.from_pretrained(config.MODEL_NAME, trust_remote_code=True)
    print(f"Model config: {cfg}")

    model = AutoModelForSequenceClassification.from_pretrained(
        config.MODEL_NAME, num_labels=len(config.LABEL_MAP)
    )

    # apply classifier dropout if provided in config
    if hasattr(config, "CLASSIFIER_DROPOUT"):
        if hasattr(model.config, "classifier_dropout"):
            model.config.classifier_dropout = config.CLASSIFIER_DROPOUT
        if hasattr(model.config, "hidden_dropout_prob"):
            model.config.hidden_dropout_prob = config.CLASSIFIER_DROPOUT

        if hasattr(model.config, "attention_probs_dropout_prob"):
            model.config.attention_probs_dropout_prob = min(
                0.15, max(0.1, config.CLASSIFIER_DROPOUT)
            )
        for m in model.modules():
            if isinstance(m, nn.Dropout):
                m.p = config.CLASSIFIER_DROPOUT
    return model, tokenizer


# Training

In [ ]:
import os
import math
import torch
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm
from torch.optim import AdamW
from huggingface_hub import login
from transformers import get_scheduler
from torch.utils.data import DataLoader
from tqdm.contrib.logging import logging_redirect_tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, accuracy_score, classification_report
from functools import partial


## train one epoch function

In [ ]:
def train_one_epoch(
    model,
    data_loader,
    loss_fn,
    optimizer,
    scheduler,
    device,
    epoch=None,
    total_epochs=None,
    gradient_accumulation_steps=1,
):
    """Huấn luyện mô hình trong một epoch bằng gradient accumulation."""
    model.train()
    total_loss = 0
    desc = f"Train" if epoch is None else f"Epoch {epoch}/{total_epochs}"
    progress_bar = tqdm(
        data_loader, desc=desc, leave=False, dynamic_ncols=True, mininterval=0.5
    )

    optimizer.zero_grad()
    steps_in_epoch = len(data_loader)
    with logging_redirect_tqdm():  # make logger calls safe
        for step, batch in enumerate(progress_bar):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # shape (batch_size, num_labels)

            loss = loss_fn(logits, labels)
            total_loss += loss.item()
            scaled_loss = loss / gradient_accumulation_steps
            scaled_loss.backward()

            if (step + 1) % gradient_accumulation_steps == 0 or (
                step + 1
            ) == steps_in_epoch:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

    return total_loss / len(data_loader)


## Evaluate

In [ ]:
def evaluate(model, data_loader, loss_fn, device):
    """Đánh giá mô hình trên tập dữ liệu."""
    model.eval()
    all_preds = []
    all_labels = []
    total_val_loss = 0

    progress_bar = tqdm(data_loader, desc="Evaluating", leave=False, dynamic_ncols=True)

    with torch.no_grad(), logging_redirect_tqdm():
        for batch in progress_bar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # <<< TÍNH LOSS TRÊN TẬP VALIDATION
            loss = loss_fn(logits, labels)
            total_val_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(data_loader)  # <<< TÍNH LOSS TRUNG BÌNH
    return all_labels, all_preds, avg_val_loss  # <<< TRẢ VỀ THÊM LOSS


# Main

In [10]:
# Tải biến môi trường từ file envs/.env.
dotenv_path = os.path.join(os.getcwd(), "envs", ".env")
load_dotenv(dotenv_path)
print(f"dotenv_path: {dotenv_path}")


dotenv_path: /home/guest/Projects/CS221/envs/.env


In [11]:
# lấy HF token để login
hf_token = os.getenv("HUGGING_FACE_TOKEN")

if hf_token:
    print("INFO: Tìm thấy HUGGING_FACE_TOKEN. Đang đăng nhập...")
    login(token=hf_token)
    print("INFO: Đăng nhập Hugging Face thành công.")
else:
    print(
        "WARNING: Không tìm thấy HUGGING_FACE_TOKEN trong file .env. Một số model có thể yêu cầu đăng nhập."
    )


INFO: Tìm thấy HUGGING_FACE_TOKEN. Đang đăng nhập...
INFO: Đăng nhập Hugging Face thành công.


## 1. Chuẩn bị dữ liệu

In [12]:
logger.info("Bắt đầu pipeline huấn luyện.")

# 1. Chuẩn bị dữ liệu
logger.info("Bước 1: Chuẩn bị dữ liệu...")
train_df, val_df = prepare_data(cfg, logger=logger)
if train_df is None:
    logger.error("Dữ liệu không thể được chuẩn bị. Dừng chương trình.")


2025-10-16 15:20:53 - [INFO] - Bắt đầu pipeline huấn luyện.
2025-10-16 15:20:53 - [INFO] - Bước 1: Chuẩn bị dữ liệu...
2025-10-16 15:20:53 - [INFO] - Chia dữ liệu: 5600 mẫu train, 1400 mẫu validation.


✅ Đọc thành công 7000 mẫu từ /home/guest/Projects/CS221/data/vihallu-train.csv

=== KIỂM TRA FORMAT DỮ LIỆU MỚI ===
Mẫu input: Câu hỏi: Vào những năm 1960, nơi nào trở thành trung tâm của thế hệ âm nhạc beat và folk, mặc dù Jackson Pollock và Willem de Kooning đã tổ chức nhiều buổi triển lãm âm nhạc nổi tiếng tại đó? Ngữ cảnh: Vào những năm 1870, hai nhà điêu khắc Augustus Saint-Gaudens và Daniel Chester French sinh sống và làm việc gần Quảng trường. Đến những năm 1920, Công viên Quảng trường Washington được công nhận cấp quốc gia là một trung tâm của phong trào nổi loạn về nghệ thuật và đạo đức. Do đó, khuôn viên NYU tại Quảng trường Washington trở nên đa dạng và hối hả nhờ có năng lượng của cuộc sống đô thị, điều này đã dẫn đến những sự thay đổi về mặt học thuật ở NYU. Những cư dân nổi tiếng thời kỳ này bao gồm Eugene O'Neill, John Sloan, và Maurice Prendergast. Vào những năm 1930, những người theo chủ nghĩa biểu hiện trừu tượng Jackson Pollock và Willem de Kooning, cùng với những ng

## 2. Tải model và tokenizer

In [13]:
logger.info(f"Bước 2: Tải model '{cfg.MODEL_NAME}' và tokenizer...")
model, tokenizer = get_model_and_tokenizer(cfg)


2025-10-16 15:20:53 - [INFO] - Bước 2: Tải model 'MoritzLaurer/ernie-m-large-mnli-xnli' và tokenizer...


Đang tải model: MoritzLaurer/ernie-m-large-mnli-xnli
Model config: ErnieMConfig {
  "act_dropout": 0.0,
  "architectures": [
    "ErnieMForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "dtype": "float32",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "ernie_m",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "paddlenlp_version": null,
  "problem_type": "single_label_classification",
  "transformers_version": "4.57.1",
  "type_vocab_size": 16,
  "vocab_size": 250002
}



In [14]:
# %pip install torchinfo


In [15]:
from torchinfo import summary
import torch  # Đảm bảo đã import torch

logger.info("Phân tích kiến trúc mô hình bằng torchinfo...")

# --- Dùng torchinfo để hiển thị ---
# Tạo một input giả với batch_size và max_length như trong config
input_ids_example = torch.randint(
    0, tokenizer.vocab_size, (cfg.BATCH_SIZE, cfg.MAX_LENGTH)
)

# 1. Gọi summary với verbose=0 để không in ra console và lưu kết quả vào biến
#    Thêm các cột bạn muốn xem, ví dụ: 'output_size', 'num_params'
model_summary = summary(
    model,
    input_data={"input_ids": input_ids_example},
    verbose=0,  # <-- Quan trọng: Ngăn không cho tự động in
    col_names=["input_size", "output_size", "num_params", "mult_adds"],
)

# 2. Chuyển đối tượng summary thành string và đưa vào logger
logger.info(f"Kiến trúc chi tiết của mô hình:\n{str(model_summary)}")


# # (Tùy chọn) Bạn vẫn có thể in ra màn hình nếu muốn xem ngay trong notebook
# print("In summary ra màn hình notebook:")
# print(model_summary)


2025-10-16 15:20:55 - [INFO] - Phân tích kiến trúc mô hình bằng torchinfo...
2025-10-16 15:20:57 - [INFO] - Kiến trúc chi tiết của mô hình:
Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Mult-Adds
ErnieMForSequenceClassification                              --                        [2, 3]                    --                        --
├─ErnieMModel: 1-1                                           [2, 512]                  [2, 1024]                 --                        --
│    └─ErnieMEmbeddings: 2-1                                 --                        [2, 512, 1024]            --                        --
│    │    └─Embedding: 3-1                                   [2, 512]                  [2, 512, 1024]            256,002,048               512,004,096
│    │    └─Embedding: 3-2                                   [2, 512]                  [2, 512, 1024]            526,336              

## 3. Tạo Dataset và DataLoader

In [16]:
# Cell [15] - PHIÊN BẢN HOÀN CHỈNH
from torch.utils.data import DataLoader

logger.info("Bước 3: Bắt đầu xử lý và tạo DataLoader với Static Padding...")

# --- 3.1: Lấy danh sách câu và nhãn ---
train_texts = train_df["input_text"].to_list()
val_texts = val_df["input_text"].to_list()
train_labels = train_df["label_id"].to_list()
val_labels = val_df["label_id"].to_list()

# --- 3.2: Tokenize, Pad và chuyển sang Tensor cùng lúc ---
logger.info("Tokenizing và Padding toàn bộ tập train...")
train_encodings = tokenizer(
    train_texts,
    max_length=cfg.MAX_LENGTH,
    truncation=True,
    padding="max_length",
    return_attention_mask=True,
    return_tensors="pt",  # <-- THAY ĐỔI QUAN TRỌNG: Yêu cầu trả về PyTorch Tensor
)

logger.info("Tokenizing và Padding toàn bộ tập validation...")
val_encodings = tokenizer(
    val_texts,
    max_length=cfg.MAX_LENGTH,
    truncation=True,
    padding="max_length",
    return_attention_mask=True,
    return_tensors="pt",  # <-- THAY ĐỔI QUAN TRỌNG: Yêu cầu trả về PyTorch Tensor
)

# --- 3.3: Tạo Dataset từ dữ liệu đã được xử lý ---
logger.info("Tạo Dataset...")
train_dataset = PreTokenizedDataset(encodings=train_encodings, labels=train_labels)
val_dataset = PreTokenizedDataset(encodings=val_encodings, labels=val_labels)

# --- 3.4: Tạo DataLoader (KHÔNG CẦN collate_fn) ---
logger.info("Tạo DataLoader...")
train_loader = DataLoader(train_dataset, batch_size=cfg.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE)

logger.info("✅ Tạo DataLoader thành công!")


2025-10-16 15:20:57 - [INFO] - Bước 3: Bắt đầu xử lý và tạo DataLoader với Static Padding...
2025-10-16 15:20:57 - [INFO] - Tokenizing và Padding toàn bộ tập train...
2025-10-16 15:20:57 - [INFO] - Tokenizing và Padding toàn bộ tập train...
2025-10-16 15:21:01 - [INFO] - Tokenizing và Padding toàn bộ tập validation...
2025-10-16 15:21:01 - [INFO] - Tạo Dataset...
2025-10-16 15:21:01 - [INFO] - Tạo DataLoader...
2025-10-16 15:21:01 - [INFO] - ✅ Tạo DataLoader thành công!


In [17]:
gradient_accumulation_steps = max(1, cfg.GRADIENT_ACCUMULATION_STEPS)
effective_batch_size = cfg.BATCH_SIZE * gradient_accumulation_steps
logger.info(
    "Gradient accumulation steps: %s | Effective batch size: %s",
    gradient_accumulation_steps,
    effective_batch_size,
)


2025-10-16 15:21:02 - [INFO] - Gradient accumulation steps: 8 | Effective batch size: 16


### Check dataset

In [18]:
print("\n--- Kiểm tra 1 batch dữ liệu đầu vào ---")
sample_batch = next(iter(train_loader))
print("Kích thước input_ids:", sample_batch["input_ids"].shape)
print("Kích thước attention_mask:", sample_batch["attention_mask"].shape)
print("Nhãn trong batch:", sample_batch["labels"])

# Giải mã một mẫu để xem nó trông như thế nào
decoded_text = tokenizer.decode(sample_batch["input_ids"][0], skip_special_tokens=False)
print("\nMột mẫu đã được token hóa và giải mã lại:")
print(decoded_text)
print("------------------------------------------\n")
# --- KẾT THÚC BƯỚC KIỂM TRA ---



--- Kiểm tra 1 batch dữ liệu đầu vào ---
Kích thước input_ids: torch.Size([2, 512])
Kích thước attention_mask: torch.Size([2, 512])
Nhãn trong batch: tensor([2, 1])

Một mẫu đã được token hóa và giải mã lại:
[CLS] Câu hỏi: Cuộc cách mạng Baal Teshuva, vốn không có tác động nhân khẩu học nào đáng kể, đã làm cho những người Do Thái thế tục trở thành những người Do Thái sùng đạo như thế nào? Ngữ cảnh: Cũng có một xu hướng của các cuộc cách mạng được khởi xướng từ phái Do Thái giáo Chính thống để hỗ trợ và giúp đỡ những người Do Thái thế tục hướng về một bản sắc Do Thái truyền thống đậm đà bản sắc dân tộc Do Thái để giảm thiểu tỷ lệ kết hôn khác chủng tộc và việc kết hôn khác tôn giáo. Nhờ những nỗ lực của các cuộc cách mạng ấy cùng với các nhóm Do Thái khác, trong25 năm qua đã có một xu hướng (được gọi là cuộc cách mạng Baal Teshuva) đã làm cho những người Do Thái thế tục trở thành những người Do Thái sùng đạo, mặc dù các tác động về nhân khẩu học của cuộc cách mạng này chưa được điều tr

## 4. Thiết lập Huấn luyện

In [19]:
import torch  # Đảm bảo đã import torch

logger.info("Bước 4: Thiết lập môi trường huấn luyện và kiến trúc model...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- Log thông tin thiết bị (GPU/CPU) ---
logger.info(f"Sử dụng thiết bị: {device}")
if device.type == "cuda":
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    logger.info(f"✅ Tìm thấy {gpu_count} GPU(s).")
    logger.info(f"✅ Đang sử dụng GPU: {gpu_name}")
else:
    logger.warning("⚠️ Không tìm thấy GPU, sử dụng CPU. Quá trình training sẽ rất chậm.")

# --- BẮT ĐẦU PHẦN THÊM MỚI ---
# Chuyển toàn bộ kiến trúc model thành dạng string để đưa vào logger
model_architecture_string = str(model)

# Ghi log kiến trúc model
logger.info(f"Kiến trúc của mô hình:\n{model_architecture_string}")
# --- KẾT THÚC PHẦN THÊM MỚI ---

# Di chuyển model đến device đã chọn
model.to(device)


2025-10-16 15:21:02 - [INFO] - Bước 4: Thiết lập môi trường huấn luyện và kiến trúc model...
2025-10-16 15:21:02 - [INFO] - Sử dụng thiết bị: cuda
2025-10-16 15:21:02 - [INFO] - ✅ Tìm thấy 1 GPU(s).
2025-10-16 15:21:02 - [INFO] - ✅ Đang sử dụng GPU: NVIDIA GeForce RTX 5070 Ti
2025-10-16 15:21:02 - [INFO] - Kiến trúc của mô hình:
ErnieMForSequenceClassification(
  (ernie_m): ErnieMModel(
    (embeddings): ErnieMEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.02, inplace=False)
    )
    (encoder): ErnieMEncoder(
      (layers): ModuleList(
        (0-23): 24 x ErnieMEncoderLayer(
          (self_attn): ErnieMAttention(
            (self_attn): ErnieMSelfAttention(
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (k_proj): Linear(in_features=1024, out

ErnieMForSequenceClassification(
  (ernie_m): ErnieMModel(
    (embeddings): ErnieMEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.02, inplace=False)
    )
    (encoder): ErnieMEncoder(
      (layers): ModuleList(
        (0-23): 24 x ErnieMEncoderLayer(
          (self_attn): ErnieMAttention(
            (self_attn): ErnieMSelfAttention(
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.02, inplace=False)
            )
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (linear1): Linear(in_features=1024, out_features=4096,

In [20]:
optimizer = AdamW(
    model.parameters(),
    lr=cfg.LEARNING_RATE,
    weight_decay=cfg.WEIGHT_DECAY,
    eps=cfg.EPSILON,
)
num_update_steps_per_epoch = math.ceil(len(train_loader) / gradient_accumulation_steps)
num_training_steps = num_update_steps_per_epoch * cfg.EPOCHS
logger.info(
    "Scheduler will run for %s total steps (%s per epoch)",
    num_training_steps,
    num_update_steps_per_epoch,
)


2025-10-16 15:21:02 - [INFO] - Scheduler will run for 3500 total steps (350 per epoch)


- Cosine Scheduler:
    Sau khi warmup, scheduler sẽ giảm learning rate theo đường cong cosine. Ban đầu nó giảm rất chậm (giữ ở mức LR cao trong thời gian dài hơn), sau đó giảm nhanh dần và cuối cùng lại giảm rất chậm khi về gần 0.
    
    Tại sao nó hiệu quả? Ý tưởng là việc giữ learning rate ở mức cao lâu hơn giúp model có cơ hội "khám phá" và thoát khỏi các điểm tối ưu kém (local minima). Sau đó, việc giảm tốc từ từ ở cuối giúp nó tinh chỉnh một cách chính xác.

In [21]:
if cfg.TOTAL_STEP_SCALE <= 0:
    warmup_steps = 0
elif cfg.TOTAL_STEP_SCALE <= 1:
    warmup_steps = max(1, int(cfg.TOTAL_STEP_SCALE * num_training_steps))
else:
    warmup_steps = min(int(cfg.TOTAL_STEP_SCALE), num_training_steps)

scheduler = get_scheduler(
    cfg.SCHEDULER_TYPE,
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=num_training_steps,
)
logger.info("Warmup steps: %s", warmup_steps)


2025-10-16 15:21:02 - [INFO] - Warmup steps: 350


In [22]:
# Chuyển class weights từ config thành tensor và đưa lên device
if cfg.CLASS_WEIGHTS:
    logger.info("Sử dụng Class Weights & Label smoothing cho hàm loss.")
    class_weights_tensor = torch.tensor(cfg.CLASS_WEIGHTS, dtype=torch.float).to(device)
    loss_fn = torch.nn.CrossEntropyLoss(
        weight=class_weights_tensor,
        label_smoothing=cfg.LABEL_SMOOTHING,
    ).to(device)
else:
    logger.info("Sử dụng CrossEntropyLoss thông thường (không có trọng số).")
    loss_fn = torch.nn.CrossEntropyLoss().to(device)


2025-10-16 15:21:02 - [INFO] - Sử dụng Class Weights & Label smoothing cho hàm loss.


## 5. Vòng lặp Huấn luyện

In [23]:
best_macro_f1 = 0.0
patience_counter = 0  # bien dem => early stopped khi f1 ko tang them => overfitting

for epoch in range(cfg.EPOCHS):
    logger.info(f"--- Epoch {epoch + 1}/{cfg.EPOCHS} ---")

    avg_train_loss = train_one_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        scheduler,
        device,
        epoch + 1,
        cfg.EPOCHS,
        gradient_accumulation_steps=gradient_accumulation_steps,
    )
    logger.info(f"Loss trung bình trên tập train: {avg_train_loss:.4f}")

    current_lr = optimizer.param_groups[0]["lr"]
    logger.info(
        f"Current Learning Rate: {current_lr:.2e}"
    )  # Dùng định dạng khoa học e.g., 8.00e-06

    # Đánh giá trên tập validation
    logger.info("Bắt đầu đánh giá trên tập validation...")
    val_labels, val_preds, avg_val_loss = evaluate(model, val_loader, loss_fn, device)

    accuracy = accuracy_score(val_labels, val_preds)
    macro_f1 = f1_score(val_labels, val_preds, average="macro")

    logger.info(f"Validation Loss: {avg_val_loss:.4f}")
    logger.info(f"Validation Accuracy: {accuracy:.4f}")
    logger.info(f"Validation Macro-F1: {macro_f1:.4f}")

    # In classification report chi tiết
    print("\nClassification Report trên tập validation:")
    target_names = [cfg.ID2LABEL[i] for i in range(len(cfg.LABEL_MAP))]
    # print(
    #     classification_report(
    #         val_labels, val_preds, target_names=target_names, digits=4
    #     )
    # )

    # In classification report chi tiết (có thể giữ lại print hoặc log từng dòng)
    report = classification_report(
        val_labels,
        val_preds,
        target_names=[cfg.ID2LABEL[i] for i in range(len(cfg.LABEL_MAP))],
        digits=4,
    )
    logger.info(f"Classification Report trên tập validation:\n{report}")

    # Lưu lại model tốt nhất dựa trên Macro-F1
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        patience_counter = 0  # << RESET BỘ ĐẾM

        logger.info(
            f"🎉 Macro-F1 cải thiện. Đang lưu model tốt nhất vào '{cfg.MODEL_OUTPUT_DIR}'..."
        )
        if not os.path.exists(cfg.MODEL_OUTPUT_DIR):
            os.makedirs(cfg.MODEL_OUTPUT_DIR)

        model.save_pretrained(cfg.MODEL_OUTPUT_DIR)
        tokenizer.save_pretrained(cfg.MODEL_OUTPUT_DIR)
        logger.info("Lưu model thành công.")
    else:
        patience_counter += 1
        logger.warning(
            f"Macro-F1 không cải thiện. Patience: {patience_counter}/{cfg.PATIENCE_LIMIT}"
        )
        if patience_counter >= cfg.PATIENCE_LIMIT:
            logger.info("Early stopping! Dừng huấn luyện.")
            break


2025-10-16 15:21:02 - [INFO] - --- Epoch 1/10 ---


Epoch 1/10:   0%|          | 0/2800 [00:00<?, ?it/s]

2025-10-16 15:27:03 - [INFO] - Loss trung bình trên tập train: 1.5303
2025-10-16 15:27:03 - [INFO] - Current Learning Rate: 8.00e-06
2025-10-16 15:27:03 - [INFO] - Bắt đầu đánh giá trên tập validation...


Evaluating:   0%|          | 0/700 [00:00<?, ?it/s]

2025-10-16 15:27:32 - [INFO] - Validation Loss: 0.8831
2025-10-16 15:27:32 - [INFO] - Validation Accuracy: 0.6136
2025-10-16 15:27:32 - [INFO] - Validation Macro-F1: 0.6139
2025-10-16 15:27:32 - [INFO] - Classification Report trên tập validation:
              precision    recall  f1-score   support

   intrinsic     0.5345    0.5061    0.5199       490
   extrinsic     0.6929    0.7245    0.7084       461
          no     0.6101    0.6169    0.6135       449

    accuracy                         0.6136      1400
   macro avg     0.6125    0.6159    0.6139      1400
weighted avg     0.6109    0.6136    0.6120      1400

2025-10-16 15:27:32 - [INFO] - 🎉 Macro-F1 cải thiện. Đang lưu model tốt nhất vào '/home/guest/Projects/CS221/models/ernie-m-large-mnli-xnli-tuned'...


...

Evaluating:   0%|          | 0/700 [00:00<?, ?it/s]

2025-10-16 15:47:34 - [INFO] - Validation Loss: 0.9048
2025-10-16 15:47:34 - [INFO] - Validation Accuracy: 0.7371
2025-10-16 15:47:34 - [INFO] - Validation Macro-F1: 0.7380
2025-10-16 15:47:34 - [INFO] - Classification Report trên tập validation:
              precision    recall  f1-score   support

   intrinsic     0.6971    0.6857    0.6914       490
   extrinsic     0.7619    0.7289    0.7450       461
          no     0.7547    0.8018    0.7775       449

    accuracy                         0.7371      1400
   macro avg     0.7379    0.7388    0.7380      1400
weighted avg     0.7369    0.7371    0.7367      1400

2025-10-16 15:47:34 - [WARNING] - Macro-F1 không cải thiện. Patience: 1/2
2025-10-16 15:47:34 - [INFO] - --- Epoch 5/10 ---



Classification Report trên tập validation:


Epoch 5/10:   0%|          | 0/2800 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [24]:
logger.info("🏁 Quá trình huấn luyện hoàn tất.")
logger.info(
    f"Model tốt nhất với Macro-F1 = {best_macro_f1:.4f} đã được lưu tại '{cfg.MODEL_OUTPUT_DIR}'"
)


2025-10-16 15:47:51 - [INFO] - 🏁 Quá trình huấn luyện hoàn tất.
2025-10-16 15:47:51 - [INFO] - Model tốt nhất với Macro-F1 = 0.7508 đã được lưu tại '/home/guest/Projects/CS221/models/ernie-m-large-mnli-xnli-tuned'
2025-10-16 15:47:51 - [INFO] - Model tốt nhất với Macro-F1 = 0.7508 đã được lưu tại '/home/guest/Projects/CS221/models/ernie-m-large-mnli-xnli-tuned'


# Phân phối kết quả đúng/sai theo từng lớp

In [25]:
val_label_names = [cfg.ID2LABEL[label_id] for label_id in val_labels]
pred_label_names = [cfg.ID2LABEL[pred_id] for pred_id in val_preds]
evaluation_df = pd.DataFrame(
    {
        "true_label": val_label_names,
        "predicted_label": pred_label_names,
    }
)
evaluation_df["status"] = evaluation_df.apply(
    lambda row: (
        "correct" if row["true_label"] == row["predicted_label"] else "incorrect"
    ),
    axis=1,
)
distribution_table = (
    evaluation_df.groupby(["true_label", "status"])
    .size()
    .unstack(fill_value=0)
    .rename_axis(None, axis=1)
    .reset_index()
    .sort_values("true_label")
)

# 1. Thêm cột 'total' bằng cách cộng cột 'correct' và 'incorrect'
distribution_table["total"] = (
    distribution_table["correct"] + distribution_table["incorrect"]
)

# 2. Thêm cột tỉ lệ đúng (correct_rate)
distribution_table["correct_rate"] = (
    distribution_table["correct"] / distribution_table["total"]
)

# 3. Thêm cột tỉ lệ sai (incorrect_rate)
distribution_table["incorrect_rate"] = (
    distribution_table["incorrect"] / distribution_table["total"]
)

# (Tùy chọn) Format các cột tỉ lệ thành dạng phần trăm cho dễ đọc
distribution_table["correct_rate"] = distribution_table["correct_rate"].map(
    "{:.2%}".format
)
distribution_table["incorrect_rate"] = distribution_table["incorrect_rate"].map(
    "{:.2%}".format
)

# In ra bảng kết quả
logger.info(f"Phân phối kết quả trên từng lớp:\n{distribution_table.to_string()}")

# Trong notebook, dùng display() sẽ cho bảng đẹp hơn
print("Bảng phân phối kết quả trên từng lớp:")
display(distribution_table)


2025-10-16 15:47:52 - [INFO] - Phân phối kết quả trên từng lớp:
  true_label  correct  incorrect  total correct_rate incorrect_rate
0  extrinsic      336        125    461       72.89%         27.11%
1  intrinsic      336        154    490       68.57%         31.43%
2         no      360         89    449       80.18%         19.82%


Bảng phân phối kết quả trên từng lớp:


,true_label,correct,incorrect,total,correct_rate,incorrect_rate
0,extrinsic,336,125,461,72.89%,27.11%
1,intrinsic,336,154,490,68.57%,31.43%
2,no,360,89,449,80.18%,19.82%
